<a href="https://colab.research.google.com/github/pythonWolf59/PDF-Text-Extractor/blob/master/PDF_to_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y poppler-utils tesseract-ocr libtesseract-dev tesseract-ocr-ara tesseract-ocr-urd
!pip install -q pdf2image python-docx opencv-python pytesseract


In [ ]:
from google.colab import files
from pdf2image import convert_from_path
import os

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

# Convert PDF to images (at lower DPI to save memory)
images = convert_from_path(pdf_path, dpi=200, fmt='jpeg')
os.makedirs("pages", exist_ok=True)
image_paths = []

for i, img in enumerate(images):
    path = f"pages/page_{i+1}.jpg"
    img.save(path, "JPEG")
    image_paths.append(path)

import cv2

def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.fastNlMeansDenoising(img, h=30)
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    cleaned_path = img_path.replace(".jpg", "_cleaned.jpg")
    cv2.imwrite(cleaned_path, img)
    return cleaned_path

import pytesseract

def extract_text(img_path, lang="eng+urd+ara"):
    return pytesseract.image_to_string(img_path, lang=lang)

from docx import Document

doc = Document()
doc.add_heading("OCR Document Output", 0)

for i, img_path in enumerate(image_paths):
    cleaned_img = preprocess_image(img_path)
    text = extract_text(cleaned_img)

    doc.add_page_break()
    doc.add_heading(f"Page {i+1}", level=1)
    for para in text.split('\n\n'):
        if para.strip():
            doc.add_paragraph(para.strip())

output_docx = "light_ocr_output.docx"
doc.save(output_docx)

files.download(output_docx)

